In [1]:
import pandas as pd
import math
import re

In [2]:
re_han = re.compile("([\u4E00-\u9FD5]+)", re.U)

def txtcnt(txt):
    cnt = 0
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            cnt += len(blk)
    return cnt

def tokens(txt,word_len):
    words = []
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            words += ngram(blk,word_len)
    return words

def ngram(txt,word_len):
    words = []
    tmp_txt = txt
    remain_len = len(tmp_txt) - word_len
    if remain_len <= 0:
        words.append(tmp_txt)
    else:
        inx = 0
        while inx <= remain_len:
            word = tmp_txt[inx:inx+word_len]
            words.append(word)
            inx += 1
    return words

In [3]:
word_len = 4
line_inx = 1
file_name = 'comment_word_'+ str(word_len) +'.txt'
with open('comment_all.txt','r') as f \
    ,open(file_name,'w') as w:
    for line in f.readlines():
        words = tokens(line,word_len)
        for word in words:
            w.write(word + ',' + str(line_inx) +',1\n')
        line_inx += 1

In [4]:
df = pd.read_csv(file_name,header=None)
df[:2]

,0,1,2
0,难受,1,1
1,问题不在,2,1


In [5]:
idf = df.filter(items=[0,1])

In [6]:
len(idf)

51002

In [7]:
idf_df = idf.drop_duplicates()

In [8]:
idf_df1 = idf_df.groupby(by=0).count()

In [9]:
idf_df1[:5]

,1
0,
一,3
一一从中,1
一一张罚,1
一万个借,1
一万这还,1


In [10]:
tf = df.filter(items=[0,2])

In [11]:
tf_df = tf.groupby(by=0).sum()

In [12]:
tf_df[:5]

,2
0,
一,3
一一从中,1
一一张罚,1
一万个借,1
一万这还,1


In [13]:
tf_df.sort_values(by=2,ascending=False).to_csv('词频.csv')

In [14]:
tf_idf = idf_df1.join(tf_df)

In [15]:
tf_idf['idf'] = 1.0/tf_idf[1]*337.0
tf_idf['tf'] = tf_idf[2]/470.0

In [16]:
tf_idf['score'] = tf_idf['idf'] * tf_idf['tf']

In [17]:
tf_idf[:5]

,1,2,idf,tf,score
0,,,,,
一,3,3,112.333333,0.006383,0.717021
一一从中,1,1,337.000000,0.002128,0.717021
一一张罚,1,1,337.000000,0.002128,0.717021
一万个借,1,1,337.000000,0.002128,0.717021
一万这还,1,1,337.000000,0.002128,0.717021


In [18]:
tf_idf1 = tf_idf.reset_index()

In [19]:
tf_idf1[(tf_idf1[2]>=3) & (tf_idf1[1]>=3)].sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
13526,哈哈哈哈,6,14,56.166667,0.029787,1.673050
40340,运输行业,4,7,84.250000,0.014894,1.254787
22411,拒不改正,3,5,112.333333,0.010638,1.195035
39441,路政部门,3,5,112.333333,0.010638,1.195035
8104,你这样的,3,5,112.333333,0.010638,1.195035
...,...,...,...,...,...,...
15330,多数都是,3,3,112.333333,0.006383,0.717021
15471,大哥真是,12,12,28.083333,0.025532,0.717021
19317,底层人民,15,15,22.466667,0.031915,0.717021
13259,呵呵,15,15,22.466667,0.031915,0.717021


In [20]:
tf_idf1[(tf_idf1[2]>=3) & (tf_idf1[1]>=3)].sort_values(by='score',ascending=False).to_csv('score.csv',index=False)

In [21]:
tf_idf1[:1]

,0,1,2,idf,tf,score
0,一,3,3,112.333333,0.006383,0.717021
